# Gravity Modelling Exercise for GEOL 5826 (Part A)

Gravimetric studies are widely acknowledged as a useful tool for studying and modeling the distributions of geological subsurface masses. Inversion of gravity data has been practiced now for several years for geophysical imaging in both two and three dimensions. Applying 2D methods is much more efficient than 3D as the processing load is lowered in proportion to the number of parameters required to compute. Depending on the amount of apriori knowledge, different methods can be chosen for 2D gravity inversion.

In this exercise, a computational tool for the development and implementation of a published method of 2D forward and inversion modelling has been designed using the prism discretization method presented by Blakely (1996). The forward and inverse modeling and inversion code was written in Julia, an expressive programming language designed for high-performance computing which can rapid perform parameter estimation using novel parallelization schemes.

## Environment setup

Run the following code blocks to import the necessary libraries. This may take some time. 

While this loads, review the two functions in the kernel.jl file. What do you think each do?

In [ ]:
using Pkg
Pkg.status()
Pkg.add("PyPlot")

# Import Libraries
using PyPlot # Plotting
using Base.Iterators # Array operators
using Random;

# Import User Generated Libraries
include("kernel.jl"); # Forward model library
include("conjugateGradient.jl"); # C.G. solver

# Constant variables
NOISE=0.15;

## Forward modelling: a simple example
The model space is parameterized the code block below. The subsurface in the survey area needs to be considered as prismatic blocks (cells) that run parallel to geological structures; the density of each block is approximated. Weights can be assigned to each block depending on depth. In this case, we are manually setting the values of model cells to generate a dipping dike anomaly. Note, that the background density value is subtracted, to generate a model of *density anomaly* from the background, rather than absolute density, as shown in the plot.

In [ ]:
# Parameterize the model

nz = 5; # number of cells in the z direction
nx = 49; # number of cells in x direction
dz = 1125; # meters, cell size in z direction (depth)
dx = 1125; # meters, cell size in x direction (distance)
background = 2.80; # grams/cc, constant starting background density

model = ones(nz, nx) * background;
model[2,20] = 3.3; # Hard code in dipping anomaly
model[3,21] = 3.3;
model[4,22] = 3.3;
model = model .- background;  # compute density anomaly

# Plot the model
fig = figure(figsize=(nx,nz))
imshow(model .+ background)
xlabel("Distance along profile (Cells)"); ylabel("Depth (Cells)"); title("Toy Denisty Model (grams/cc)")
colorbar(orientation="horizontal")

The forward modeling operator, or kernel (G) is the mathematical relationship between the model parameters that describe the subsurface (such as density variations) and the predicted geophysical response that would be measured at the surface. The Julia script kernel.jl contains the functions that assemble this operator given the abobe arguments describing the model space. Run the code block below to generate the G operator.

In [ ]:
# Assemble a sensitivity matrix, or kernel, (G operator) with assuming no change in datum height (z)

M = size(model)[1] * size(model)[2];
N = size(model)[2];

G = generateKernel(M, zeros(N), dx, dz);

Estimated data can be computed using the relationship 
$$d_{\text{est}} = Gm$$
where $m$ is the true model. Run the code block below to estimate and plot data from the synthetic model.

In [ ]:
# Forward model data
fv=vec(model');
d_est = G * fv;

# Plot the values of d_est using PyPlot
fig = figure(figsize=(nx/2,nz/2))
plot(d_est)
xlabel("Station"); ylabel("Gravity (mgal)");
title("Gravity anomaly over profile");
show()

## Inversion

We will now estimate model parameters based on observed data by utilizing techniques of linear inverse theory. 

$$m_{\text{est}} = G^{-1} d_{obs}$$


This inversion code is a adaptation of the Backus‐Gilbert approach to inversion that finds an acceptable model nearest to the initial estimate. An iterative application of the technique allowed a density model to be developed at a modest expense. In order to constrain the inversion minimum distance, smoothness and compactness weights were included within an iterative Lagrangian formulation. Apriori information on density and the density range allowed for the survey area.

For this example, the estimated data values from the above forward model, are to be used at the observed values (with added noise) for the following inversion

In [ ]:
d_obs = d_est;  # In this case, copy Observed Values directly from the Estimated

# Fix random numbers for reproducability
Random.seed!(44); println("Random Seed No. '",randstring(),"'\n") 

d_obs = d_obs .+ randn(length(d_obs)) * NOISE; # Add noise to the 'observed' data

# Plot the values of d_est using PyPlot
fig = figure(figsize=(nx/2,nz/2))
plot(d_obs)
xlabel("Station"); ylabel("Gravity (mgal)"); title("Gravity anomaly over profile with noise")
show()

Various settings for the constrained gravity inversion (ζ, β, ε) are found below. Please see Vatankhah et. al. (2014) for more information. These can be adjusted to provide depth, compactness and smoothness contraints on the model. The number of iterations for inversion is also defined below.

In [ ]:
#SET PARAMETERS
ζ = 0.10;
β = 0.50;
ε = 0.99;
iterations=5;

Linear algebra below. Matrices described in Vatankhah et. al. (2014) are assembled here, including the G operator.

In [ ]:
# Get dimensions
M = size(model)[1] * size(model)[2];
N = size(model)[2];

# generate a G with assuming no change in datum height (z)
G = generateKernel(M, zeros(N), dx, dz);

g_obs = d_obs; # Can add padding here
# Make an H matrix "flatness"
H = Matrix{Float64}(I,M,M) * -1; #smoothness matrix
for i = 1:M-1, j = 2:M
    H[i,i+1] = 1;
end

# Make A matrix [G]
#               [H]

A = hcat(G', ζ * H')';

# Make a P matrix ("hard constraint")
P = Matrix{Float64}(I, M, M);

# Make a Q matrix ("depth weighting matrix")
Q = Matrix{Float32}(I, M, M);

for j = 1:M
    Q_jj = 1 / (((j*dz) + ε)^β);
    Q[j,:] = Q[j,:] * Q_jj;
end

# Make a V matrix ("compactness")
𝓥 = Matrix{Float64}(I,M,M); #initially set to Identity

# Set any initial density anomaly estimates
ρ_back = zeros(M) # Background anomaly is zero

ρ = Array{Float64}(undef,M) .+ ρ_back;
ρ_min = -1;
ρ_max = 1;

null_vector = zeros(M) # Null vector

for i = 1:M # Update hard constraint
    if ρ[i] != 2.5
        P[i,i] = 1e-2
    end
end

The minimization of the objective function is below. Inversion is accomplished using the conjugate gradient method for solving systems of equations.Running the following code block will compute and plot a vector of model estimates, as well as the estimated data from this updated model, and the mean squared error from the observed data. This may take more time to run dependant on the number of iterations and model parameters.

In [ ]:
for i = 1:iterations
    W = inv(P) * Q * 𝓥
    g_cal = G * ρ
    b = vcat(g_obs - g_cal, null_vector);
    Θ = return_m((A * inv(W)) * transpose((A * inv(W))), b, 40); # C.G.

    global ρ = ρ + (inv(W) * transpose(A * inv(W)) * Θ)

    for j = 1:M
        𝓥[j,j] = 1 / ((ρ[j])^2 + 1e-11)
        if ρ[j] < ρ_min || ρ[j] > ρ_max
            P[j,j] = 1e-2
        else
            P[j,j] = 1
        end
    end
end


d_estf = G * ρ
mse = sum((d_obs .- d_estf).^2) / length(d_obs) # Calculate Mean Squared Error
print("The mean squared error betweent he observed and estimated data is: ", round(mse, digits=5))
# Plot the values of d_est using PyPlot
fig = figure(figsize=(nx/2,nz/2))
plot(d_estf, label="Estimated Data")
plot(d_obs, label="Observed Data")
xlabel("Station")
ylabel("Gravity (mgal)")
legend()
title("Gravity anomaly over profile")
show()

model_est = reshape(ρ, (nx, nz)) .+ background

fig = figure(figsize=(nx/2,nz/2))
imshow(transpose(model_est) , vmin=2.8, vmax=3.3)
colorbar(orientation="horizontal")

## End
Created by Christopher Mancuso. 2024.
Rev 1.05